In [121]:
import matplotlib.pyplot as plt 
import pandas as pd
import torch.nn as nn
import torch
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
import shap


In [122]:
df1 = pd.read_csv('ready_data.csv') 

In [123]:
import torch.nn as nn

class TimeSeriesModel(nn.Module):
    def __init__(self, input_size, hidden_size=128, output_size=2, num_layers=1):
        super(TimeSeriesModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        # out = out[:, -1, :]
        out = self.dropout(out)
        out = self.fc(out)
        return out


In [124]:
model = joblib.load("overfit_mitigated/xgb_earmy_stopping.pkl")

In [125]:
# ✅ Save the model weights only
# torch.save(model.state_dict(), 'try4/model_weights.pt')


In [126]:
df1.columns

Index(['pid', 'datatime', 'sbp', 'dbp', 'temperature', 'conductivity', 'uf',
       'blood_flow', 'gender', 'birthday', 'first_dialysis', 'DM', 'keyindate',
       'dialysisstart', 'dialysisend', 'weightstart', 'weightend', 'dryweight',
       'pulse', 'respiratory_rate', 'blood_oxygen_lvl', 'glucose_lvl',
       'hypotension', 'age', 'dialyzer', 'bath', 'technique', 'gain',
       'bath_temperature', 'replacement_Volume', 'kt', 'Bath_Flow',
       'bicarbonate_conductivity', 'arterial_Pressure', 'Venous_Pressure',
       'transmembrane_Pressure', 'avg_weekly_sessions', 'Dialysis_Duration',
       'De_hour', 'De_minutes', 'Ds_hour', 'Ds_minutes', 'session_year',
       'session_month', 'session_dayofweek', 'session_is_weekend', 'year',
       'month', 'dayofweek', 'is_weekend', 'hour', 'minutes', 'fd_year',
       'fd_month'],
      dtype='object')

In [127]:

# from ydata_profiling import ProfileReport

In [128]:

# # Create the profile
# profile = ProfileReport(df1, title="Dataset Profiling Report", explorative=True)

# # To display it in a Jupyter Notebook:
# # profile.to_notebook_iframe()

# # OR to save it to an HTML file:
# profile.to_file("dataset_profile.html")

In [129]:
test_df= df1[[
    'dbp',
 'temperature',
 'conductivity',
 'gender',
 'birthday',
 'weightstart',
 'weightend',
 'dryweight',
 'age',
 'gain',
 'bath_temperature',
 'replacement_Volume',
 'kt',
 'bicarbonate_conductivity',
 'Venous_Pressure',
 'transmembrane_Pressure',
 'avg_weekly_sessions',
 'session_year',
 'fd_year',
 'fd_month']]
    # ['sbp', 'dbp', 'temperature', 'conductivity', 'uf',
    #    'blood_flow', 'gender', 'birthday', 'DM',
    #     'weightstart', 'weightend', 'dryweight',
    #    'pulse', 'respiratory_rate', 'blood_oxygen_lvl', 'glucose_lvl',
    #    'hypotension', 'age', 'dialyzer', 'bath', 'technique', 'gain',
    #    'bath_temperature', 'replacement_Volume', 'kt', 'Bath_Flow',
    #    'bicarbonate_conductivity', 'arterial_Pressure', 'Venous_Pressure',
    #    'transmembrane_Pressure',
    #     'Ds_hour', 'Ds_minutes', 'session_year',
    #    'session_month', 'session_dayofweek', 'session_is_weekend', 'year',
    #    'month', 'dayofweek', 'is_weekend', 'hour', 'minutes', 'fd_year',
    #    'fd_month']]



# df1[['sbp','dbp', 'temperature', 'birthday',
#        'weightstart', 'weightend', 'dryweight', 'glucose_lvl','age', 'dialyzer',
#        'bath_temperature', 'kt','arterial_Pressure','avg_weekly_sessions', 'session_year', 'session_month','fd_year', 'fd_month','year','month']]

In [130]:
test_df.shape

(48392, 20)

In [131]:
test_df.sample(1).to_csv('sample.csv')

In [132]:
Y = pd.DataFrame( df1['Dialysis_Duration'])

In [133]:
# Y['session'] = df1[ 'avg_weekly_sessions']

In [134]:
Y

,Dialysis_Duration
0,240.0
1,240.0
2,240.0
3,240.0
4,240.0
...,...
48387,240.0
48388,240.0
48389,240.0
48390,240.0


In [135]:
# fig, axes = plt.subplots(11, 4, figsize=(20, 20))

# axes = axes.flatten()
# i=0
# for column in test_df.columns:
    
#     ax = axes[i]
#     i+=1
#     print(column)
#     ax.scatter(test_df[column], Y.Dialysis_Duration)
#     ax.set_xlabel(column)
#     ax.set_ylabel("Dialysis_Duration")
#     ax.set_title(f'cluster : {column}')
   
#     # print(cluster_df.head(1))
#     # Number of features in the dataset



# # Adjust layout for better spacing
# plt.tight_layout()
# plt.show()

In [136]:
# fig, axes = plt.subplots(11, 4, figsize=(30, 30))

# axes = axes.flatten()
# i=0
# for column in test_df.columns:
    
#     ax = axes[i]
#     i+=1
#     print(column)
#     ax.scatter(test_df[column], Y.session)
#     ax.set_xlabel(column)
#     ax.set_ylabel("Dialysis_Duration")
#     ax.set_title(f'cluster : {column}')
   
#     # print(cluster_df.head(1))
#     # Number of features in the dataset



# # Adjust layout for better spacing
# plt.tight_layout()
# plt.show()

In [137]:
Y.Dialysis_Duration.unique()

array([240., 243., 259., 230., 233., 242., 232., 236., 229., 231., 246.,
       237., 235., 248., 273., 239., 217., 245., 238., 244., 203., 210.,
       216., 180., 251., 223., 218., 250., 241., 228., 234., 220., 214.,
       247., 227., 222., 249., 120., 226., 213., 252., 258., 254., 205.,
       300., 253., 224., 268., 270., 219., 221., 190., 262., 202., 260.,
       184., 256., 540., 211., 225., 207., 185., 212., 850., 263., 200.,
       204., 168., 174., 206., 188., 191., 171., 182., 183., 208., 215.,
       209., 255., 155., 167., 162., 153., 106., 150., 156., 144., 170.,
       141., 165., 124., 122., 169., 157., 151., 197., 179., 140., 154.,
       145., 186., 181., 277., 281., 269., 267., 264., 257., 261., 292.,
       279., 265., 346., 283., 280., 266., 275.,  60., 282., 315., 295.,
       291., 193., 360., 271., 840., 272., 199., 285.])

In [138]:
X_train, X_test, y_train, y_test=train_test_split(test_df, Y, test_size=0.1, random_state=8)

In [139]:
# model.score(X_test,y_test)

In [140]:
# score = model.score(X_test, y_test)
# print(f"- R² score on test data: {score:.4f}")
# rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
# print(f"- RMSE score on test data: {rmse:.4f}")

In [141]:
from sklearn.preprocessing import StandardScaler

# # scaler = StandardScaler()
# scaler =joblib.load("last try 2\scalerX.pkl")
# scaler_y =joblib.load("last try 2\scalerY.pkl")

X_train_scaled = X_train
# scaler.transform(X_train)
X_val_scaled = X_test
# scaler.transform(X_test)
Y_train_scaled = y_train
# scaler_y.transform(y_train)
Y_val_scaled = y_test
# scaler_y.transform(y_test)


In [142]:
from torch.utils.data import DataLoader, TensorDataset
# Create datasets and loaders
import torch

# /*************  ✨ Windsurf Command ⭐  *************/
# X_train_scaled = X_train_scaled.values
# X_val_scaled = X_val_scaled.values
# /*******  f52b8168-83a6-4692-b086-21b9d381d329  *******/
X_train_test = torch.tensor(X_train_scaled.values, dtype=torch.float32)
y_train_test = torch.tensor(Y_train_scaled.values, dtype=torch.float32)

train_dataset = TensorDataset(X_train_test, y_train_test)


X_val_test = torch.tensor(X_val_scaled.values, dtype=torch.float32)
y_val_test = torch.tensor(Y_val_scaled.values, dtype=torch.float32)

val_dataset = TensorDataset(X_val_test, y_val_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [143]:
# /*************  ✨ Windsurf Command 🌟  *************/
# model.val()

# def get_model_features(model):
#     """Get the feature names input to a model."""
#     if hasattr(model, 'feature_names_in_'):
#         return model.feature_names_in_
#     elif hasattr(model, 'feature_names_'):
#         return model.feature_names_
#     else:
#         raise ValueError('Model does not have feature names attribute.')
# /*******  73c4631c-43b3-4abd-bf4c-840c351681c0  *******/

# /*************  ✨ Windsurf Command ⭐  *************/
# # Get the feature names from the model
# feature_names = model.get_booster().feature_names
# /*******  16ddaab3-2662-43ae-87d6-55c231cb04f8  *******/


len(model.get_booster().feature_names)


20

In [144]:
test_df.columns

Index(['dbp', 'temperature', 'conductivity', 'gender', 'birthday',
       'weightstart', 'weightend', 'dryweight', 'age', 'gain',
       'bath_temperature', 'replacement_Volume', 'kt',
       'bicarbonate_conductivity', 'Venous_Pressure', 'transmembrane_Pressure',
       'avg_weekly_sessions', 'session_year', 'fd_year', 'fd_month'],
      dtype='object')

In [145]:
# model.eval()
with torch.no_grad():
    
    X_val_test = torch.tensor(X_val_scaled.values, dtype=torch.float32)
    print( X_val_test)
    preds = model.predict(X_test)
    # preds = scaler_y.inverse_transform(preds)  # Convert back to real values
    y_val_test = torch.tensor(y_test.values, dtype=torch.float32)
    actuals =y_test
    

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("MAE:", mean_absolute_error(actuals, preds))
print("RMSE:", np.sqrt(mean_squared_error(actuals, preds)))
print("R² Score:", r2_score(actuals, preds))


tensor([[7.6000e+01, 3.6000e+01, 1.4000e+01,  ..., 2.0130e+03, 2.0110e+03,
         1.0000e+00],
        [5.7000e+01, 3.7000e+01, 1.3900e+01,  ..., 2.0180e+03, 2.0040e+03,
         2.0000e+00],
        [7.2000e+01, 3.6500e+01, 1.4100e+01,  ..., 2.0140e+03, 2.0140e+03,
         2.0000e+00],
        ...,
        [6.0000e+01, 3.6500e+01, 1.4000e+01,  ..., 2.0160e+03, 2.0120e+03,
         6.0000e+00],
        [5.0000e+01, 3.6000e+01, 1.4100e+01,  ..., 2.0170e+03, 2.0020e+03,
         9.0000e+00],
        [6.3000e+01, 3.6000e+01, 1.4700e+01,  ..., 2.0150e+03, 2.0130e+03,
         5.0000e+00]])
MAE: 0.4298628728251812
RMSE: 2.147489642087348
R² Score: 0.9589750170707703


In [146]:
actuals

,Dialysis_Duration
4132,241.0
6120,240.0
39356,240.0
19658,240.0
22983,240.0
...,...
2693,240.0
6639,240.0
40594,240.0
24601,240.0


In [147]:
preds

array([240.66438, 240.20535, 240.13   , ..., 240.1355 , 240.76378,
       240.4559 ], dtype=float32)

In [148]:
# from sklearn.metrics import mean_absolute_error, r2_score

# model.eval()
# test_preds = []
# test_targets = []

# with torch.no_grad():
#     for X_batch, y_batch in val_loader:  # Assuming you have a test DataLoader
#         outputs = model(X_batch)
#         test_preds.append(outputs.cpu())
#         test_targets.append(y_batch.cpu())

# test_preds = torch.cat(test_preds).numpy()
# test_targets = torch.cat(test_targets).numpy()

# mae = mean_absolute_error(test_targets, test_preds)
# r2 = r2_score(test_targets, test_preds)

# print(f"Test MAE: {mae:.4f}, Test R²: {r2:.4f}")


In [149]:
# test_preds


In [150]:
# test_targets

In [151]:
# import shap
# import torch
# from torch.utils.data import DataLoader

# # Assuming your model and DataLoader are ready
# # For example, let's assume you have a trained model and a dataloader (e.g., test_loader)
# # Your model should be in eval mode to avoid unnecessary changes during inference
# model.eval()

# # Take one batch from your dataloader (let's assume it's a single batch)
# # Make sure you take one sample, not the whole batch (if you want to explain individual predictions)
# data_iter = iter(val_loader)
# inputs, labels = next(data_iter)

# # Get a single sample to explain (batch size = 1)
# sample_input = inputs[0].unsqueeze(0)  # Add batch dimension

# # Create a SHAP explainer for your model using DeepExplainer (for PyTorch models)
# # Here we use a small batch from the dataset to initialize the explainer
# background_data = inputs[:100]  # You can use a few samples from the dataset for the background
# explainer = shap.Explainer(model, background_data)

# # Now calculate SHAP values for the single sample
# single_shap_value = explainer.shap_values(sample_input)


In [152]:
# model.get_booster().

In [153]:
# # Example: grab a few samples from the training dataloader
# background_samples = [X_test.values]
# # for batch in train_loader:  # assuming your DataLoader is called train_loader
# #     inputs, _ = batch
# #     background_samples.append(inputs)
# #     if len(background_samples) >= 2:  # Take 2 batches, for example
# #         break

# # # background = torch.cat(X_train, dim=0)  # Concatenate into one tensor
# # background = torch.cat([x.to(next(model.get_booster()).device) for x in background_samples], dim=0)
# # background = background.to(next(model.get_booster()).device)  # Move to same device as model

# # /*************  ✨ Windsurf Command ⭐  *************/
# background = torch.cat(background_samples, dim=0)  # Concatenate into one tensor
# background = background.to(next(model.parameters()).device)  # Move to same device as model

# /*************  ✨ Windsurf Command ⭐  *************/
# # Fix for XGBRegressor which doesn't have a `parameters()` method
# if hasattr(model, 'parameters'):
#     background = background.to(next(model.parameters()).device)
# else:
#     background = background.to('cpu')  # Default to CPU if not found
# /*******  4a1768cf-dab9-49b7-8152-590628caddc5  *******/


background_samples = [torch.tensor(x, dtype=torch.float32) for x in X_test.values]

# Concatenate tensors into one tensor
background = torch.cat(background_samples, dim=0)

# Move to the same device as the model
background = background.to('cpu')


In [154]:
background.shape

torch.Size([96800])

In [155]:
background

tensor([  76.,   36.,   14.,  ..., 2015., 2013.,    5.])

In [156]:
# Example: grab a few samples from the test dataloader
# test_samples = []
# for batch in val_loader:  # assuming your test DataLoader is called test_loader
#     inputs, _ = batch
#     test_samples.append(inputs)
#     if len(test_samples) >= 1:  # Take 1 batch, for example
#         break

# test_samples = torch.cat(test_samples, dim=0)
# test_samples = test_samples.to(next(model.parameters()).device)




test_samples = [torch.tensor(x, dtype=torch.float32) for x in X_test.values]

# Concatenate tensors into one tensor
test_samples = torch.cat(test_samples, dim=0)

# Move to the same device as the model
test_samples = background.to('cpu')



In [158]:
# Explainability setup
# /*************  ✨ Windsurf Command ⭐  *************/
# # GradientExplainer currently only works with PyTorch or TensorFlow models
# # If you have an XGBoost model, you can use the TreeExplainer instead
# if isinstance(model, xgboost.sklearn.XGBRegressor):
#     explainer = shap.TreeExplainer(model)
# else:
#     explainer = shap.GradientExplainer (model, background)
# /*******  5b4673b0-4b8d-49c9-a730-ee9e1c6f0bdc  *******/
test_samples = test_samples.reshape(test_samples.shape[0], -1)
# test_samples = test_samples.reshape(-1, test_samples.shape[-1])  # reshape to 2D
explainer = shap.TreeExplainer (model)  # use a small sample for background
shap_values = explainer.shap_values(test_samples, check_additivity=False)     # explain on a few test samples

# /*************  ✨ Windsurf Command ⭐  *************/
#         test_samples = test_samples.reshape(-1, test_samples.shape[-1])  # reshape to 2D
# /*******  6155c70c-bf42-4fb3-ba3a-d489c42235bb  *******/


XGBoostError: [23:40:57] C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api_utils.h:129: Check failed: std::accumulate(shape.cbegin(), shape.cend(), static_cast<bst_ulong>(1), std::multiplies<>{}) == chunksize * rows (193600 vs. 2032800) : 

In [ ]:
# Feature names
feature_names = ['sbp', 'dbp', 'temperature', 'conductivity', 'uf',
       'blood_flow', 'gender', 'birthday', 'DM',
        'weightstart', 'weightend', 'dryweight',
       'pulse', 'respiratory_rate', 'blood_oxygen_lvl', 'glucose_lvl',
       'hypotension', 'age', 'dialyzer', 'bath', 'technique', 'gain',
       'bath_temperature', 'replacement_Volume', 'kt', 'Bath_Flow',
       'bicarbonate_conductivity', 'arterial_Pressure', 'Venous_Pressure',
       'transmembrane_Pressure',
        'Ds_hour', 'Ds_minutes', 'session_year',
       'session_month', 'session_dayofweek', 'session_is_weekend', 'year',
       'month', 'dayofweek', 'is_weekend', 'hour', 'minutes', 'fd_year',
       'fd_month']  # replace with your real feature names

# Plot summary of feature importances
shap.summary_plot(shap_values[:,:,0], test_samples,plot_type="bar",plot_size=0.2, feature_names=feature_names)


In [ ]:

shap.summary_plot(shap_values[:,:,1], test_samples,plot_type="bar",plot_size=0.2, feature_names=feature_names)

In [ ]:
# shap.decision_plot(explainer.expected_value[1], shap_values[1], feature_names)

In [ ]:
import shap

# Pick a single patient
patient_idx = 0  # for the first patient
patient_input = test_samples
print(patient_input.shape)
# Compute SHAP values for just that one patient
single_shap_value = explainer.shap_values(patient_input, check_additivity=False)
print(single_shap_value[0].shape)

shap_values_for_class = single_shap_value[0][:, patient_idx]
print(shap_values_for_class.shape)
# Plot the force plot
# shap.initjs()



In [ ]:
shap.initjs()
shap.force_plot(
    explainer.expected_value[0], 
    shap_values_for_class,  # ← single sample
    patient_input[0].cpu().numpy(),  # ← features for that sample
    feature_names=feature_names
)

In [ ]:
patient_input.cpu().numpy().shape

In [ ]:
# Extract SHAP values for class 0 (shape should be (n_samples, n_features))
shap_values_class_0 = shap_values[0]  # Assuming shape (1, 44, 2)

# For a single sample, select the first sample's SHAP values for class 0
shap_values_for_plot = shap_values[0:32, :, 0]  # Shape (1, 44)

# Get the corresponding features (must be 2D)
features_for_plot = patient_input.cpu().numpy() # Shape (1, 44)

# Now plot the dependence plot
shap.dependence_plot(
    43,  # Feature index to plot
    shap_values_for_plot, 
    features_for_plot,
    feature_names=feature_names,

)

In [ ]:


# Create a 2x2 grid for subplots
fig, axes = plt.subplots(11, 4, figsize=(20, 20))

axes = axes.flatten()

for i in range (len(feature_names)):
    # print(f"Cluster {cluster_id} DataFrame:")
    ax = axes[i]
    shap.dependence_plot(
    i,  # Feature index to plot
    shap_values_for_plot, 
    features_for_plot,
    feature_names=feature_names,
    ax=axes[i],
    show=False

)
    # Number of features in the dataset



# Adjust layout for better spacing
plt.tight_layout()
plt.show()

In [ ]:
background.shape

In [ ]:
len(feature_names)

In [ ]:
import torch
from lime import lime_tabular

# Define prediction function
def predict_fn(x):
    x_tensor = torch.tensor(x, dtype=torch.float32)
    with torch.no_grad():
        outputs = model(x_tensor)
        probs = torch.softmax(outputs, dim=1)
    return probs.numpy()

back = pd.DataFrame(background , columns=feature_names)
# Create explainer
explainer = lime_tabular.LimeTabularExplainer(
    training_data=back.values,
    feature_names=back.columns.tolist(),
    mode='regression'
)



In [ ]:

# Explain a prediction
i = 20  # Example index
test_sc=scaler.inverse_transform(test_samples)
test = pd.DataFrame(test_sc , columns=feature_names)

exp = explainer.explain_instance(
    data_row=test.iloc[i].values,
    predict_fn=predict_fn,
    num_features=20
)

exp.show_in_notebook()

In [ ]:
test

In [ ]:
# shap.initjs()
# shap.plots.waterfall(shap_values[:,:,0])  # First sample, first class

In [ ]:
type(shap_values[1,1])

In [ ]:
# shap.plots.waterfall(
#     shap_values[0],  # SHAP values for the sample and class
#     max_display=12,  # Limit the number of features shown
#     show=True        # Display the plot
# )

In [ ]:
# print("Shape of patient_input[0]:", patient_input[0].cpu().numpy().shape)
# print("Shape of single_shap_value[0][0]:", single_shap_value[0][0].shape)
# print("Length of feature_names:", len(feature_names))


In [ ]:
# print("Model output for a patient:", model(patient_input).shape)


In [ ]:
# import joblib

# # Sauvegarder le modèle
# joblib.dump(model, 'last try 2\\cnn_try_data_final sampling_50k 96.pkl')
# print("Modèle enregistré sous le nom 'random_forest_model.pkl'.")

In [ ]:
# from sklearn.model_selection import KFold, cross_val_score
# from sklearn.metrics import mean_squared_error, make_scorer
# # Use KFold with 5 splits
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# # Use negative RMSE as a scoring function (because sklearn is annoying)
# neg_rmse = make_scorer(mean_squared_error, greater_is_better=False)

# # cross_val_score returns NEGATIVE MSE values, so we negate and sqrt for RMSE
# scores = cross_val_score(model, test_df, Y, cv=kf, scoring=neg_rmse)
# rmse_scores = np.sqrt(-scores)

# print("RMSE scores:", rmse_scores)
# print("Average RMSE:", np.mean(rmse_scores))

In [ ]:
# from sklearn.metrics import mean_squared_error, r2_score

# # Assume you've trained the model
# train_preds = model.predict(X_train)
# test_preds = model.predict(X_test)

# train_rmse = mean_squared_error(y_train, train_preds, squared=False)
# test_rmse = mean_squared_error(y_test, test_preds, squared=False)

# train_r2 = r2_score(y_train, train_preds)
# test_r2 = r2_score(y_test, test_preds)

# print(f"Train RMSE: {train_rmse}, Test RMSE: {test_rmse}")
# print(f"Train R²: {train_r2}, Test R²: {test_r2}")


In [ ]:
# from sklearn.model_selection import learning_curve
# import matplotlib.pyplot as plt

# train_sizes, train_scores, test_scores = learning_curve(
#     model, test_df, Y, cv=5, scoring='neg_mean_squared_error'
# )

# train_rmse = -train_scores.mean(axis=1) 
# test_rmse = -test_scores.mean(axis=1) 

# plt.figure(figsize=(10, 6))
# plt.plot(train_sizes, train_rmse.mean(axis=1), 'o-', label='Training RMSE', color='blue')
# plt.plot(train_sizes, test_rmse.mean(axis=1), 'o-', label='Validation RMSE', color='orange')

# plt.legend()
# plt.xlabel("Training set size")
# plt.ylabel("RMSE")
# plt.title("Learning Curve")
# plt.show()
